In [2]:
import torch
from transformers import AutoTokenizer, AutoModel
from scipy.spatial.distance import cosine
import pandas as pd
import numpy as np
from collections import Counter
import time


def parse_embedding_from_str(str_emb):
    emb = []
    for part in str_emb.strip()[1:-1].split(" "):
        fixed_part = part.strip()
        if len(fixed_part) > 0:
            emb.append(float(fixed_part))

    return emb


def load_embeddings(path):
    df = pd.read_csv(path)
    all_embs = []
    all_key_skills = []
    for _, row in df.iterrows():  # tqdm(df.embedding):
        all_embs.append(parse_embedding_from_str(row["embedding"]))
        all_key_skills.append(eval(row["key_skills"]))

    df["embedding"] = all_embs
    df["key_skills"] = all_key_skills

    return df


class EmbeddingExtractor:
    def __init__(self, model, tokenizer, initial_df, similarity_metric=cosine):
        self.embeddings = {}
        self.model = model
        self.tokenizer = tokenizer
        self.similarity_metric = similarity_metric

        self._initialize_embeddings(initial_df)

    def _initialize_embeddings(self, initial_df):
        for _, row in initial_df.iterrows():
            name = row["name"]
            emb = row["embedding"]

            self.embeddings[name] = emb

    def extract(self, text):
        if text in self.embeddings:
            return self.embeddings[text]

        encoded_input = self.tokenizer(
            [text], padding=True, truncation=True, max_length=64, return_tensors="pt"
        )
        with torch.no_grad():
            model_output = self.model(**encoded_input)
        embedding = model_output.pooler_output[0].numpy().astype(np.float32)

        self.embeddings[text] = embedding

        return embedding

    def extract_batch(self, texts):
        encoded_input = self.tokenizer(
            texts, padding=True, truncation=True, max_length=64, return_tensors="pt"
        )
        with torch.no_grad():
            model_output = self.model(**encoded_input)

        for i in range(len(texts)):
            text = texts[i]
            embedding = model_output.pooler_output[i]

            self.embeddings[text] = embedding

    def similarity(self, emb1, emb2):
        return 1 - self.similarity_metric(emb1, emb2)


class VacancyFinder:
    def __init__(self, embedding_extractor, initial_df):
        self.embedding_extractor = embedding_extractor
        self.titles = {}
        self.vacancy_stats_by_title = {}

        self._load_vacancies(initial_df)

    def _load_vacancies(self, initial_df):
        titles = list(set(initial_df.parent.values))

        for title in titles:
            self.titles[title] = self.embedding_extractor.extract(title)
            self.vacancy_stats_by_title[title] = []
            for _, row in initial_df.iterrows():
                name = row["name"]
                key_skills = row["key_skills"]
                id_ = row["id"]

                self.vacancy_stats_by_title[title].append(
                    [name, key_skills, id_, self.embedding_extractor.extract(name)]
                )

    def _select_best_titles(self, emb, amount):
        stats = []
        for title_name, title_emb in self.titles.items():
            stats.append(
                [
                    title_name,
                    title_emb,
                    self.embedding_extractor.similarity(emb, title_emb),
                ]
            )

        return sorted(stats, key=lambda x: -x[-1])[:amount]

    def _select_best_vacancies(self, emb, titles, amount):
        stats = []
        for title in titles:
            for (
                vacancy_name,
                key_skills,
                vacancy_id,
                vacancy_emb,
            ) in self.vacancy_stats_by_title[title]:
                stats.append(
                    [
                        vacancy_name,
                        key_skills,
                        vacancy_id,
                        vacancy_emb,
                        title,
                        self.embedding_extractor.similarity(emb, vacancy_emb),
                    ]
                )

        return sorted(stats, key=lambda x: -x[-1])[:amount]

    def get_best_vacancies(self, vacancy_name, nearest_titles=3, amount=20):
        emb = self.embedding_extractor.extract(vacancy_name)
        titles = self._select_best_titles(emb, nearest_titles)
        title_names = [title_name for title_name, _, _ in titles]

        best_vacancies = self._select_best_vacancies(emb, title_names, amount)
        return best_vacancies


def aggregate_skills(all_skills):
    skills_list = []
    for key_skills in all_skills:
        skills_list.extend(key_skills)

    return Counter(skills_list)


class SkillsExtractor:
    def __init__(
        self,
        path_to_vacancies_info,
        model_path="/Users/dl/GitHub/RecSysApp/models/LaBSE-en-ru",
        tokenizer_path="/Users/dl/GitHub/RecSysApp/models/LaBSE-en-ru",
    ):
        tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
        model = AutoModel.from_pretrained(model_path)
        emb_df = load_embeddings(path_to_vacancies_info)
        embedding_extractor = EmbeddingExtractor(model, tokenizer, emb_df)

        self.vacancy_finder = VacancyFinder(embedding_extractor, emb_df)

    def key_skills_for_profession(
        self, profession, max_skills=100, min_frequency=3, nearest_vacancies=50
    ):
        all_key_skills = []
        for _, key_skills, _, _, _, _ in self.vacancy_finder.get_best_vacancies(
            profession, amount=nearest_vacancies
        ):
            all_key_skills.append(key_skills)

        selected_skills = []
        for name, amount in aggregate_skills(all_key_skills).most_common():
            if len(selected_skills) >= max_skills or amount < min_frequency:
                break
            else:
                selected_skills.append(name)

        return selected_skills

start_time = time.time()

skills_extractor = SkillsExtractor(
    path_to_vacancies_info="/Users/dl/GitHub/RecSysApp/data/Вакансии/Vacancy.csv"
)

end_time = time.time()
elapsed_time = end_time - start_time

print(f"Время выполнения: {elapsed_time:.6f} секунд")

Время выполнения: 952.293136 секунд


In [27]:
start_time = time.time()

sk = skills_extractor.key_skills_for_profession("Программист")

end_time = time.time()
elapsed_time = end_time - start_time

print(sk)

print(f"Время выполнения: {elapsed_time:.6f} секунд")

AttributeError: 'SkillsExtractor' object has no attribute 'key_skills_for_profession'

Время выполнения (до загрузки модели включительно): 0.193772 секунд

Время выполнения (до чтения csv включительно): 11.957095 секунд
key_skills = "['Cisco', 'MikroTik', 'TCP/IP', 'Asterisk', 'CUCM']" --- str
embedding = '[ 3.10295783e-02 -7.58733869e-01 -1.50719807e-01  2.28532985' --- str

Время выполнения (до преобразования key_skills и embedding в list включительно): 74.685128 секунд
key_skills = ['Cisco', 'MikroTik', 'TCP/IP', 'Asterisk', 'CUCM'] --- list
embedding = [0.0310295783, -0.758733869, -0.150719807, 0.228532985, ...] --- list

Время выполнения (до преобразования получения словаря вакансия=эмбединги (схлопываются вакансии) включительно): 160.669755 секунд
Было вакансий 164065 ---- стало 79399

In [1]:
import torch
from transformers import AutoTokenizer, AutoModel
from scipy.spatial.distance import cosine
import pandas as pd
import numpy as np
from collections import Counter
import time

def parse_embedding_from_str(str_emb):
    emb = []
    for part in str_emb.strip()[1:-1].split(" "):
        fixed_part = part.strip()
        if len(fixed_part) > 0:
            emb.append(float(fixed_part))

    return emb


def load_embeddings(path):
    df = pd.read_csv(path)
    all_embs = []
    all_key_skills = []
    for _, row in df.iterrows():
        all_embs.append(parse_embedding_from_str(row["embedding"]))
        all_key_skills.append(eval(row["key_skills"]))

    # Конвертер строк в списки
    df["embedding"] = all_embs
    df["key_skills"] = all_key_skills

    return df


class EmbeddingExtractor:
    def __init__(self, model, tokenizer, initial_df, similarity_metric=cosine):
        self.embeddings = {}
        self.model = model
        self.tokenizer = tokenizer
        self.similarity_metric = similarity_metric

        self._initialize_embeddings(initial_df)

    def _initialize_embeddings(self, initial_df):
        for _, row in initial_df.iterrows():
            name = row["name"]
            emb = row["embedding"]

            # Словарь их название вакансии -> эмбединг
            self.embeddings[name] = emb

    def extract(self, text):
        # Название группы совпадает с минимум одной вакансией
        if text in self.embeddings:
            return self.embeddings[text]

        encoded_input = self.tokenizer(
            [text], padding=True, truncation=True, max_length=64, return_tensors="pt"
        )
        with torch.no_grad():
            model_output = self.model(**encoded_input)
        embedding = model_output.pooler_output[0].numpy().astype(np.float32)

        # Добавление названия группы и ее эмбедингов в названия вакансий
        self.embeddings[text] = embedding

        return embedding


class VacancyFinder:
    def __init__(self, embedding_extractor, initial_df):
        self.embedding_extractor = embedding_extractor
        # Название группы вакансий -> ее эмбединги
        self.titles = {}
        self.vacancy_stats_by_title = {}

        self._load_vacancies(initial_df)

    def _load_vacancies(self, initial_df):
        # Группы вакансий
        titles = list(set(initial_df.parent.values))

        # Проход по всем группам
        for title in titles:
            self.titles[title] = self.embedding_extractor.extract(title)


class SkillsExtractor:
    def __init__(
        self,
        path_to_vacancies_info,
        model_path="/Users/dl/GitHub/RecSysApp/models/LaBSE-en-ru",
        tokenizer_path="/Users/dl/GitHub/RecSysApp/models/LaBSE-en-ru",
    ):
        self.tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
        self.model = AutoModel.from_pretrained(model_path)
        self.emb_df = load_embeddings(path_to_vacancies_info)
        self.embedding_extractor = EmbeddingExtractor(self.model, self.tokenizer, self.emb_df)

        self.vacancy_finder = VacancyFinder(self.embedding_extractor, self.emb_df)

start_time = time.time()

skills_extractor = SkillsExtractor(
    path_to_vacancies_info="/Users/dl/GitHub/RecSysApp/data/Вакансии/Vacancy.csv"
)

end_time = time.time()
elapsed_time = end_time - start_time

print(f"Время выполнения: {elapsed_time:.6f} секунд")

Время выполнения: 99.992805 секунд


In [3]:
len(skills_extractor.embedding_extractor.embeddings)

79444

In [6]:
len(skills_extractor.vacancy_finder.titles["Специалист по тендерам"])

768

In [9]:
vacancy_stats_by_title = {}

for title, v in skills_extractor.vacancy_finder.titles.items():
    vacancy_stats_by_title[title] = []
    for _, row in skills_extractor.emb_df.iterrows():
        name = row["name"]
        key_skills = row["key_skills"]
        id_ = row["id"]

        vacancy_stats_by_title[title].append(
            [name, key_skills, id_, skills_extractor.embedding_extractor.extract(name)]
        )

In [20]:
len(vacancy_stats_by_title)

99

In [21]:
len(vacancy_stats_by_title["Специалист по тендерам"])

164065

In [22]:
len(vacancy_stats_by_title["Специалист по тендерам"][0])

4

In [24]:
len(vacancy_stats_by_title["Специалист по тендерам"][0][3])

768

In [27]:
vacancy_stats_by_title["Аналитик"] == vacancy_stats_by_title["Специалист по тендерам"]

True

In [ ]:
релевантность и полезность и видбек

In [89]:
type(skills_extractor.embedding_extractor.embeddings)

dict

In [104]:
class Timer(ContextDecorator):
    """Контекстный менеджер для замера времени выполнения кода."""
    def __enter__(self):
        self.start = time.time()
        return self

    def __exit__(self, *args):
        self.end = time.time()
        print(f"Время выполнения: {self.end - self.start:.6f} секунд")

class VacancyFinder:
    def __init__(self, skills_extractor):
        self.embedding_extractor = skills_extractor.embedding_extractor
        self.titles = {} # Название группы и эмбединги
        self.vacancy_stats_by_title = {}

        self._load_vacancies(skills_extractor.emb_df)

    def _load_vacancies(self, initial_df):
        # код берет все значения столбца parent, удаляет дубликаты и возвращает их в виде списка уникальных значений
        titles = list(set(initial_df.parent.values))

        # Проходим по каждой группе вакансий
        for title in titles:
            self.titles[title] = extract(
                title,
                skills_extractor.embedding_extractor.embeddings,
                skills_extractor.tokenizer,
                skills_extractor.model
            )

with Timer():
    vacancy_finder = VacancyFinder(skills_extractor)

    # print(len(vacancy_finder.titles['Комплаенс-менеджер']))

Время выполнения: 1.901957 секунд


In [ ]:
skills_extractor.embedding_extractor.embeddings

In [102]:
skills_extractor.tokenizer(
    ["Специалист по тендерам"], padding = True, truncation = True, max_length = 64, return_tensors = "pt"
)['token_type_ids']

torch.Size([1, 8])

In [105]:
sum(1 for value in vacancy_finder.titles.values() if value is not None)

99

In [93]:
len(skills_extractor.embedding_extractor.embeddings['Специалист по тендерам'])

768

In [81]:
print(list(set(skills_extractor.emb_df.parent.values)))

['Комплаенс-менеджер', 'Руководитель отдела логистики', 'Системный инженер', 'Фотограф, ретушер', 'Бухгалтер', 'Геодезист', 'Менеджер ресторана', 'Бизнес-аналитик', 'Инженер по эксплуатации', 'Администратор', 'Арт-директор, креативный директор', 'Агент по недвижимости', 'Финансовый директор (CFO)', 'Видеооператор, видеомонтажер', 'Архитектор', 'Специалист по тендерам', 'Методолог', 'Менеджер продукта', 'Делопроизводитель, архивариус', 'Руководитель отдела аналитики', 'Менеджер по персоналу', 'Руководитель филиала', 'Юрист', 'Менеджер по туризму', 'Специалист по взысканию задолженности', 'Маркетолог-аналитик', 'Руководитель строительного проекта', 'Финансовый контролер', 'Технический писатель', 'Оператор ПК, оператор базы данных', 'BI-аналитик, аналитик данных', 'Координатор отдела продаж', 'Специалист по подбору персонала', 'Специалист службы безопасности', 'Менеджер по закупкам', 'Директор по информационным технологиям (CIO)', 'DevOps-инженер', 'Копирайтер, редактор, корректор',

In [58]:
len(skills_extractor.embedding_extractor.embeddings)

79399

In [59]:
len(skills_extractor.emb_df)

164065

In [46]:
skills_extractor.emb_df['key_skills'][0]

['Cisco', 'MikroTik', 'TCP/IP', 'Asterisk', 'CUCM']

In [47]:
skills_extractor.emb_df['embedding'][0][:10]

[0.0310295783,
 -0.758733869,
 -0.150719807,
 0.228532985,
 -0.397263587,
 -0.403417349,
 -0.455173582,
 -0.0370771214,
 0.0684766769,
 -0.228751957]

In [82]:
skills_extractor.vacancies.select("parent").unique().to_list()

AttributeError: 'SkillsExtractor' object has no attribute 'vacancies'